In [6]:
# get applicationids that have active video feedback (application settings page → SAM db)
# convert each applicationId to the correct guid
import cx_Oracle
import uuid

dsn_tns = cx_Oracle.makedsn('sit1oracle.prodlabs.local', '1521', service_name='GV')

def get_oracleid_from_bytes(byteString):
    return str(uuid.UUID(bytes=byteString).hex).upper()

# reference https://stackoverflow.com/questions/37684714/oracle-raw-in-string-format-to-standard-guid
def raw_to_guid(rawString):
    split_groups = [rawString[idx:idx + 2] for idx in range(0, len(rawString), 2)]

    return (split_groups[3]+split_groups[2]+split_groups[1]+split_groups[0]+'-'+split_groups[5]+split_groups[4]+'-'+split_groups[7]+
    split_groups[6]+'-'+split_groups[8]+split_groups[9]+'-'+split_groups[10]+split_groups[11]+split_groups[12]+split_groups[13]+split_groups[14]+split_groups[15])

try:
    with cx_Oracle.connect(user='sam', password='Password1', dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            cursor.execute("select id from lk_settings where name='videoFeedback.enabled'")
            while True:
                row = cursor.fetchone()
                setting_id = get_oracleid_from_bytes(row[0])
                break
            print(setting_id)
            application_ids = []
            cursor.execute("select applicationid from applicationsettings where settingid = :settingid and value='true'", [setting_id])
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                application_id = get_oracleid_from_bytes(row[0])
                application_id_guid = raw_to_guid(application_id)
                application_ids.append(application_id_guid)
            print(application_ids)

except cx_Oracle.Error as error:
    print(error)


721145BD0084164DABA880EA0C4938A9
['3EE14C3D-4494-4EB6-BB12-AC86015FAF0C', '105D7CB1-3262-486F-BB54-AC600119E814', '8FB5C42A-0F4C-45B6-8131-A783012F0225', '62218C0A-454C-4C17-BD00-A793011F1517', 'DFC50B9F-CBBC-4E14-9AD3-A89B0117EA38', 'C91F5450-C315-41EE-8DC7-A9B3014A2278', '3585FAED-8866-4701-9549-A77F000984E9', 'EAC21315-3F00-429E-9DA2-AADB011F4BE3', '1E469A8D-1AC4-4077-8641-A7C20118CE43', '943E761E-27FC-4A96-8384-A6F300131523', 'C6F182D9-4489-48C1-8EA6-A7BD00399FE9', '5880BD64-2A6C-43C6-B562-A7CB016A221B', '13B92C1B-E809-46B0-AF08-A7E6010DC54D', 'D419A700-6FE7-47BD-9958-AB04000846CD', 'BD82443B-8212-4D0C-8ABE-ACB0015C8192', '6D871792-735D-497E-9B23-AC8A014C6600', 'D2343865-7D95-4898-9B47-ACCA012B6C2E', 'F0202742-1197-4035-A3C5-AC2C01503624', '9EA9B384-3C69-4B74-93F0-AD0301641714', '3DF03C59-222F-439F-A006-A97500E41814', 'A3DDFD13-3773-45CA-AD0F-AC67013288AE', '7F8D3B4E-9F36-4286-B3BC-A8F201188FAC', '51F864DE-0570-4E77-8F52-ABF400F0BB66', '4E1F70C7-372E-4531-A902-A64F017B738F', '7B329

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
# get panelids corresponding to these applicationids (CPM db)
import pyodbc
import pandas as pd

cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=listener.icm-sit1-sqlserver-ee.cncjmkwlkqyp.us-west-2.rds.amazonaws.com;"
            "Database=cpm_sit;"
            "UID=cpm_user;"
            "PWD=Password1;")
cnxn = pyodbc.connect(cnxn_str)

panel_ids = []
tenant_ids = []
for appl_id in application_ids:
    data = pd.read_sql("select Id, TenantId from Panels where applicationId='{}'".format(appl_id), cnxn)
    panel_id = data['Id'].iloc[0]
    tenant_id = data['TenantId'].iloc[0]
    panel_ids.append(panel_id)
    tenant_ids.append(tenant_id)
print(panel_ids)
print(tenant_ids)

['F4586A56-D693-474A-856D-AC86015FACE5', '89F70F3C-527F-4D61-9A6F-AC600119E6AF', '8F2B9893-DD25-432E-8717-A78301277374', 'FFD53877-D44E-4CBB-B3A3-A793011CD5A9', '5E0147B3-0C43-4E68-A5D4-A89B0117E93F', '564A7A44-3BAF-4BB1-99C5-A9B3014A2179', '8D8EF15A-9840-4CFE-82BF-A77F0002BFB6', '93A5E3AE-89D5-4471-9CDB-AADB011F4A3D', '68E08BA6-3C8B-4424-B1B3-A7C20115BB2B', '2812063D-9024-4EDC-8B0D-A6FB0011930F', '816E3569-D05A-4404-9721-A7BD0135B327', 'B3324CB9-0C9D-4B65-9E9A-A7CB0165C248', '544334B0-6864-4118-9619-A7E6010CB321', 'A3E15E55-30D7-410E-A908-AB0400084500', '5C175646-7B26-4541-BB80-ACB0015C7FC9', '1F1361FE-407C-4BF9-9238-AC8A014C642E', '2298B611-CB61-4D1F-B5CB-ACCA012B6A19', '8BA4283C-0380-413A-877C-AC2C01503374', '00276831-09C6-47C5-915F-AD0301641541', '7FBCF420-375B-4478-9A93-A97500E41704', 'E775F163-EDC4-49F1-93C7-AC67013285DE', '8CD4F154-7612-4D4B-8614-A8F201188F5E', 'DE5841DC-6B41-4D0D-8C4B-ABF400F0B90E', 'D9FD0731-B344-4321-8712-D20D1C087524', '4EAB339B-A5A2-4FDA-9B0C-ABB20133062C',

In [ ]:
# generate list of memberIds who have unsubscribed (CPM db)
def get_purged_members_query(panel_id):
	return """\
	DECLARE @panelId AS uniqueidentifier = '{}';

	WITH PanelMembers AS (
		SELECT * FROM [cpm_sit].[dbo].Members WHERE panelId = @panelId
	)

	SELECT Id FROM PanelMembers
	WHERE
		MemberStatusSequenceId IN (
			SELECT mvv.SequenceId
			FROM [cpm_sit].[dbo].MemberVariableValues mvv
			JOIN [cpm_sit].[dbo].MemberVariables mv
				ON mvv.MemberVariableId = mv.Id
			WHERE
				mv.Name = 'memberstatus'
				AND mvv.[Key] = 'purged'
				AND mv.PanelId = @panelId
				AND mvv.PanelId = @panelId
		)
		AND MemberIdentity1SequenceId IN (
			SELECT mvv.SequenceId
			FROM [cpm_sit].[dbo].MemberVariableValues mvv
			JOIN [cpm_sit].[dbo].MemberVariables mv
				ON mvv.MemberVariableId = mv.Id
			WHERE
				mv.Name = 'email'
				AND mvv.TextValue LIKE '%@purged.disabled'
				AND mv.PanelId = @panelId
				AND mvv.PanelId = @panelId
		)""".format(panel_id)

panel_to_members = dict()

for panel_id in panel_ids:
	purgedMembersQuery = get_purged_members_query(panel_id)
	data = pd.read_sql(purgedMembersQuery, cnxn)
	panel_to_members[panel_id] = data['Id'].values.tolist()

print(panel_to_members)

In [ ]:
# convert tenant ids to oracle guid

# reference https://stackoverflow.com/questions/68886687/java-guid-convert-into-oracle-raw16-string 
def guid_to_raw(guidString):
    guidString = guidString.replace("-", "")
    split_groups = [guidString[idx:idx + 2] for idx in range(0, len(guidString), 2)]

    return (split_groups[3]+split_groups[2]+split_groups[1]+split_groups[0]+split_groups[5]+split_groups[4]+split_groups[7]+
    split_groups[6]+split_groups[8]+split_groups[9]+split_groups[10]+split_groups[11]+split_groups[12]+split_groups[13]+split_groups[14]+split_groups[15])

# generate list of respondentIds who have responded to and completed VPM (VCSD db)
dsn_tns = cx_Oracle.makedsn('sit1oracle.prodlabs.local', '1521', service_name='GMASTER')

tenant_to_respondents = dict()

try:
    with cx_Oracle.connect(user='sys', password='Password1', dsn=dsn_tns, mode=cx_Oracle.SYSDBA) as conn:
        with conn.cursor() as cursor:
            for tenant_id in tenant_ids:
                tenant_id = guid_to_raw(tenant_id)
                respondent_ids = []
                cursor.execute("""select distinct Resp.respondentid from firefly_rsp.response Resp inner join firefly_rsp.stepresponsedata StepResp on Resp.responseid = StepResp.responseid where StepResp.questiontype = 'VideoFeedbackPlugin' 
                and Resp.tenantid = :tenantid and Resp.completedat is not NULL and Resp.respondentid is not NULL""", [tenant_id])
                while True:
                    row = cursor.fetchone()
                    if row is None:
                        break
                    respondent_id = row[0]
                    respondent_ids.append(respondent_id)
                tenant_to_respondents[tenant_id] = respondent_ids

except cx_Oracle.Error as error:
    print(error)

print(tenant_to_respondents)


In [ ]:
# map respondent ids to member ids
tenant_to_members = dict()

dsn_tns = cx_Oracle.makedsn('sit1oracle.prodlabs.local', '1521', service_name='GMASTER')

try:
    with cx_Oracle.connect(user='sys', password='Password1', dsn=dsn_tns, mode=cx_Oracle.SYSDBA) as conn:
        with conn.cursor() as cursor:
            for tenant_id in tenant_to_respondents:
                member_ids = []
                for respondent_id in tenant_to_respondents[tenant_id]:
                    cursor.execute("select publicid from firefly_app.respondent where respondentid= :respondentId", [respondent_id])
                    while True:
                        row = cursor.fetchone()
                        public_id = row[0].split(':')[2]
                        break
                    member_ids.append(public_id) 
                tenant_to_members[tenant_id] = member_ids

except cx_Oracle.Error as error:
    print(error)

print(tenant_to_members)


In [39]:
# intersect the lists
from itertools import chain

def flatten_dict_values(dict):
    return [x.lower() for x in chain.from_iterable(dict.values())]


def intersect_lists(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

cpm_members = flatten_dict_values(panel_to_members)

vcsd_members = flatten_dict_values(tenant_to_members)

print(intersect_lists(cpm_members, vcsd_members))

['794ed14a-775a-4376-93c4-ac4c0161bd93']
